<a href="https://colab.research.google.com/github/sylver86/Spam-Detection-System-NLP/blob/main/Progetto_Spam_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Progetto : Sistema di Spam Detection

## Panoramica del Progetto

L'obiettivo di questo progetto è creare una libreria Python capace di analizzare le email ricevute. La libreria fornirà funzionalità per classificare le email come SPAM o NON SPAM, identificare i principali argomenti tra le email SPAM, calcolare la distanza semantica tra questi argomenti ed estrarre le organizzazioni menzionate nelle email NON SPAM.

### Obiettivi

1. **Addestrare un Classificatore per Identificare le Email SPAM**
   - Utilizzare il dataset fornito per addestrare un modello di machine learning in grado di classificare accuratamente le email come SPAM o NON SPAM.
   - Valutare le prestazioni del classificatore utilizzando metriche standard come accuratezza, precisione, recall e F1-score.

2. **Individuare i Principali Argomenti tra le Email SPAM**
   - Effettuare il topic modeling sulle email SPAM per scoprire i principali temi o argomenti presenti.
   - Utilizzare tecniche come la Latent Dirichlet Allocation (LDA) o la Non-negative Matrix Factorization (NMF) per l'estrazione degli argomenti.

3. **Calcolare la Distanza Semantica tra gli Argomenti**
   - Misurare la distanza semantica tra gli argomenti individuati per comprendere l'eterogeneità degli stessi.
   - Applicare metriche come la Similarità Coseno o la Distanza di Jaccard per valutare la differenza tra i temi.

4. **Estrarre le Organizzazioni dalle Email NON SPAM**
   - Utilizzare tecniche di Named Entity Recognition (NER) per identificare e estrarre i nomi delle organizzazioni menzionate nelle email NON SPAM.
   - Valutare l'accuratezza dell'estrazione confrontando i risultati con un set di dati annotato manualmente.

### Struttura del Progetto

1. **Preprocessing dei Dati**
   - Pulizia del dataset e preparazione dei dati per il training del modello.
   - Normalizzazione del testo, rimozione di stop words, tokenizzazione e stemming/lemmatizzazione.

2. **Addestramento del Classificatore**
   - Sperimentazione con diversi algoritmi di machine learning (es. Naive Bayes, Support Vector Machines, Random Forest, o reti neurali).
   - Ottimizzazione degli iperparametri e valutazione del modello migliore.

3. **Topic Modeling**
   - Applicazione di LDA o NMF per identificare i principali argomenti nelle email SPAM.
   - Interpretazione dei risultati e visualizzazione degli argomenti.

4. **Calcolo della Distanza Semantica**
   - Implementazione di metodi per calcolare la distanza semantica tra i vari argomenti.
   - Analisi dell'eterogeneità degli argomenti identificati.

5. **Named Entity Recognition**
   - Utilizzo di librerie come spaCy o NLTK per estrarre le organizzazioni dalle email NON SPAM.
   - Valutazione dell'accuratezza dell'estrazione tramite confronto con un set di dati di riferimento.


## Preprocessing dei Dati

Iniziamo a vedere il dataset per l'avvio delle procedure di pre-processing dei dati.

In [28]:
import pandas as pd
import spacy
from nltk.corpus import stopwords
import re
import string
import nltk

nltk.download('stopwords')

# Ignora le righe corrotte
df = pd.read_csv("https://raw.githubusercontent.com/sylver86/Spam-Detection-System-NLP/main/spam_dataset_.csv", on_bad_lines='skip')
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\nthe transport v...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\nhpl ...,0
5168,2933,ham,Subject: calpine daily gas nomination\n>\n>\nj...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


Come possiamo vedere alcuni testi riportano etichettatura 0 altri 1 a seconda se il testo viene classificato come spam o meno.

Stiamo parlando quindi di creare un classificatore binario supervisionato.

Iniziamo a pulire il testo attraverso le seguenti attività di Data Cleaning:

1. Lowercase
2. Rimozione punteggiatura
3. Lemmatizzazione
4. Rimozione Stop words
5. Rimozione numeri e spazi multipli

Per far questo andiamo a crearci un metodo di pulizia sotto riportato utilizzando il modello in inglese dal momento che il testo contenuto in analisi è in lingua inglese:

In [29]:
english_stopwords = stopwords.words('english')

nlp = spacy.load('en_core_web_sm')
punctuaction = set(string.punctuation) #con il set evitiamo ripetizioni

def data_cleaner(sentence):
        sentence = sentence.lower() # Effettuiamo un lowercase della frase
        for c in string.punctuation:
            sentence = sentence.replace(c, " ") # Rimuoviamo la punteggiatura
        document = nlp(sentence)
        sentence = ' '.join(token.lemma_ for token in document) # Effettuiamo la lemmatizzazione
        sentence = ' '.join(word for word in sentence.split() if word not in english_stopwords) #Effettuiamo la rimozione stopwords
        sentence = re.sub('\d','',sentence) # Eliminiamo eventuali numeri
        return sentence

Eseguiamo dunque tale processo per tutto il dataset andando ad aggiornare la colonna "text":

In [30]:
df['text_cleaning'] = df['text'].apply(data_cleaner)

Verifichiamo la nuova colonna quale output mostrerà in seguito all'attività di DataCleaning che abbiamo appena effettuato.

In [31]:
df

,Unnamed: 0,label,text,label_num,text_cleaning
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0,subject enron methanol meter follow note I gi...
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0,subject hpl nom january see attached file hp...
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0,subject neon retreat ho ho ho around wonderful...
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1,subject photoshop window office cheap main tre...
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0,subject indian spring deal book teco pvr reven...
...,...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\nthe transport v...,0,subject put ft transport volume decrease c...
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\nhpl ...,0,subject follow nom hpl take extra mmcf wee...
5168,2933,ham,Subject: calpine daily gas nomination\n>\n>\nj...,0,subject calpine daily gas nomination julie I m...
5169,1409,ham,Subject: industrial worksheets for august 2000...,0,subject industrial worksheet august activity ...


## Addestramento di un Classificatore

Ora che il Dataset è pronto andremo a valutare alcuni classificatori per poi scegliere il migliore in termini di performance e quindi di metriche.

Iniziamo a dividere la parte di train da quella di test con :

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text_cleaning'], df['label_num'], test_size=0.25, random_state=42)

Andiamo ora a creare una vettorizzazione TF-IDF della colonna riferita al train e al test.

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Creazione del vettorizzatore TF-IDF e trasformazione dei dati
vectorizer = TfidfVectorizer(binary=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

Adesso possiamo procedere con l'addestramento di un classificatore. Quelli che andremo a mettere a confronto saranno :

1. Naive Bayes
2. Support Vector Machines (SVM)
3. Reti neurali

### Naive Bayes

Proviamo ad utilizzare il Bernoulli Naive Bayes.

In [34]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report

# Addestramento del classificatore Bernoulli Naive Bayes
clf = BernoulliNB()
clf.fit(X_train_tfidf, y_train)

# Predizione sui dati di test
y_pred = clf.predict(X_test_tfidf)

Proviamo a valutare le prestazioni del modello.

In [35]:
# Valutazione delle prestazioni del modello
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8275328692962104
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89       930
           1       0.88      0.45      0.59       363

    accuracy                           0.83      1293
   macro avg       0.85      0.71      0.74      1293
weighted avg       0.84      0.83      0.81      1293



### Support Vector Machines (SVM)

Andiamo ora a vedere si applica invece il modello SVM al problema di classificazione.

In [36]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train_tfidf, y_train)

# Predizione sui dati di test
y_pred = svc.predict(X_test_tfidf)


In [37]:
# Valutazione delle prestazioni del modello
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9860788863109049
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       930
           1       0.96      0.99      0.98       363

    accuracy                           0.99      1293
   macro avg       0.98      0.99      0.98      1293
weighted avg       0.99      0.99      0.99      1293



### Reti Neurali

In [38]:
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Multi-layer Perceptron Classifier (MLP)
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,))
mlp_clf.fit(X_train_tfidf, y_train)
y_pred = mlp_clf.predict(X_test_tfidf)

In [39]:
# Valutazione delle prestazioni del modello
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9899458623356535
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       930
           1       0.98      0.98      0.98       363

    accuracy                           0.99      1293
   macro avg       0.99      0.99      0.99      1293
weighted avg       0.99      0.99      0.99      1293



Come possiamo vedere sia il modello SVM che a Reti Neurali dimostrano le migliori performance predittive.

# Individuazione dei Topic principali su SPAM

Come altro task previsto nel progetto andiamo adesso a individuare i Principali Argomenti tra le Email SPAM nel dettaglio andremo a:

    Effettuare il topic modeling sulle email SPAM per scoprire i principali temi o argomenti presenti.
    Utilizzare tecniche come la Latent Dirichlet Allocation (LDA) o la Non-negative Matrix Factorization (NMF) per l'estrazione degli argomenti.



Quello che faremo sarà dunque l'applicazione dell'algoritmo **Latent Dirichlet Allocation** LDA che permette di individuare l'insieme di argomenti trattati.

Iniziamo a importarci gensim

In [40]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

Ci andiamo a creare il dizionario di tutte le parole contenute nello SPAM

Andiamo quindi a tokenizzare il testo utilizzando gensim.corpora

In [41]:
def sent_to_words(items):
    for item in items:
        yield(simple_preprocess(item, deacc=True)) # Viene restituita una lista (con deacc=True elimina la punteggiatura dal testo)


df = df[df['label_num']==1] # Vado a filtrarmi solo i record di tipo Spam
data_words = list(sent_to_words(df['text_cleaning'])) # Tokenizziamo il testo

Vediamo quale è il risultato della Tokenizzazione per la prima frase del dataset

In [42]:
data_words[1]

['subject',
 'look',
 'medication',
 'good',
 'source',
 'difficult',
 'make',
 'material',
 'condition',
 'well',
 'good',
 'law',
 'easy',
 'enough',
 'ruin',
 'bad',
 'law',
 'excuse',
 'find',
 'good',
 'simpli',
 'site',
 'medication',
 'net',
 'perscription',
 'easy',
 'delivery',
 'private',
 'secure',
 'easy',
 'well',
 'see',
 'rightly',
 'pound',
 'week',
 'squint',
 'million',
 'get',
 'anything',
 'ever',
 'want',
 'erection',
 'treatment',
 'pill',
 'anti',
 'depressant',
 'pill',
 'weight',
 'loss',
 'http',
 'splicing',
 'bombahakcx',
 'com',
 'knowledge',
 'human',
 'power',
 'synonymous',
 'high',
 'quality',
 'stuff',
 'low',
 'rate',
 'moneyback',
 'guarantee',
 'god',
 'nature',
 'sufficeth',
 'unto',
 'wise',
 'hath',
 'need',
 'author']

Come abbiamo potuto vedere, abbiamo tokenizzato le frasi del nostro documento ora per poter dare in pasto al modello LDA occorre andare a vettorizzare. Come prima cosa prima di processare la vettorizzazione andiamo a definirci il nostro dizionario.

In [43]:
id2word = corpora.Dictionary(data_words)

Andiamo adesso a vettorizzarlo il che vuol dire andare a sostituire le parole in vettori numerici:

In [44]:
corpus = [id2word.doc2bow(text) for text in data_words] # Il metodo doc2bow effettua il bag of word tramite il "Count Vectorizer"

In [45]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1)]

Come possiamo vedere abbiamo la vettorizzazione pronta possiamo quindi procedere a impostare il modello LDA:

In [50]:
from pprint import pprint
import gensim

# Inizializziamo con 10 i topics che vogliamo nel nostro modello - qui ad esempio mettiamo 10 TOPICS
num_topics = 10

# Creo il modello LDA utilizzando LdaMulticore, che è efficiente per il calcolo su più core della CPU
lda_model = gensim.models.LdaMulticore(
    corpus=corpus,  # 'corpus' è una collezione di documenti testuali in forma vettoriale che abbiamo processato in BoW e CountVectorizer
    id2word=id2word,  # 'id2word' è un dizionario che mappa gli ID numerici delle parole ai loro testi
    num_topics=num_topics,  # Numero di topic desiderati nel modello
    passes=10,  # Numero di passaggi
    chunksize=100,  # Dimensione dei chunk per migliorare l'efficienza
    workers=4,  # Numero di core da utilizzare
    alpha='asymmetric',  # Ottimizzazione automatica di alpha
    eta='auto'  # Ottimizzazione automatica di eta
)

# Stampa il primo topic. È necessario specificare l'indice del topic da stampare.
pprint(lda_model.print_topics(num_topics=10, num_words=10))

# Applica il modello LDA al corpus per ottenere la distribuzione dei topic per ogni documento.
doc_lda = lda_model[corpus]


[(0,
  '0.017*"subject" + 0.005*"http" + 0.004*"com" + 0.004*"www" + '
  '0.003*"content" + 0.003*"paliourg" + 0.003*"html" + 0.003*"gr" + '
  '0.003*"get" + 0.003*"die"'),
 (1,
  '0.017*"subject" + 0.009*"get" + 0.008*"com" + 0.008*"http" + 0.006*"money" '
  '+ 0.005*"new" + 0.005*"time" + 0.005*"please" + 0.005*"email" + '
  '0.005*"want"'),
 (2,
  '0.025*"pill" + 0.013*"subject" + 0.011*"viagra" + 0.009*"cialis" + '
  '0.009*"prescription" + 0.007*"drug" + 0.007*"mg" + 0.007*"good" + '
  '0.007*"get" + 0.006*"order"'),
 (3,
  '0.018*"company" + 0.010*"statement" + 0.009*"stock" + 0.007*"information" + '
  '0.007*"may" + 0.006*"report" + 0.006*"security" + 0.006*"investment" + '
  '0.006*"price" + 0.005*"within"'),
 (4,
  '0.017*"price" + 0.011*"subject" + 0.008*"software" + 0.006*"save" + '
  '0.006*"adobe" + 0.006*"xp" + 0.005*"microsoft" + 0.005*"retail" + '
  '0.005*"http" + 0.004*"professional"'),
 (5,
  '0.030*"font" + 0.028*"td" + 0.023*"nbsp" + 0.021*"height" + 0.020*"http" +

Ottenuto il modello LDA andiamo adesso a mostrare le parole chiave per ciascun topic:

In [51]:
topics = lda_model.print_topics(num_topics=num_topics, num_words=10)
print("Topics:")
pprint(topics)

Topics:
[(0,
  '0.017*"subject" + 0.005*"http" + 0.004*"com" + 0.004*"www" + '
  '0.003*"content" + 0.003*"paliourg" + 0.003*"html" + 0.003*"gr" + '
  '0.003*"get" + 0.003*"die"'),
 (1,
  '0.017*"subject" + 0.009*"get" + 0.008*"com" + 0.008*"http" + 0.006*"money" '
  '+ 0.005*"new" + 0.005*"time" + 0.005*"please" + 0.005*"email" + '
  '0.005*"want"'),
 (2,
  '0.025*"pill" + 0.013*"subject" + 0.011*"viagra" + 0.009*"cialis" + '
  '0.009*"prescription" + 0.007*"drug" + 0.007*"mg" + 0.007*"good" + '
  '0.007*"get" + 0.006*"order"'),
 (3,
  '0.018*"company" + 0.010*"statement" + 0.009*"stock" + 0.007*"information" + '
  '0.007*"may" + 0.006*"report" + 0.006*"security" + 0.006*"investment" + '
  '0.006*"price" + 0.005*"within"'),
 (4,
  '0.017*"price" + 0.011*"subject" + 0.008*"software" + 0.006*"save" + '
  '0.006*"adobe" + 0.006*"xp" + 0.005*"microsoft" + 0.005*"retail" + '
  '0.005*"http" + 0.004*"professional"'),
 (5,
  '0.030*"font" + 0.028*"td" + 0.023*"nbsp" + 0.021*"height" + 0.020*

In questo modo abbiamo una rappresentazione delle 10 parole piu rappresentative dei 10 topic individuati all'interno del dataset di Spam.

# Calcolo distanza semantica tra i topics ottenuti

Una volta definiti i topics andiamo dunque a verificare quale è la distanza semantica tra i topics ottenuti andando quindi ad estrarre le distribuzioni di probabilità dei topic e calcolandovi la distsanza coseno :

In [60]:
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

# Estrazione delle distribuzioni di probabilità dei topic
topic_word_distributions = lda_model.get_topics()

# Calcolo della distanza coseno tra tutte le coppie di topic
distance_matrix = cosine_distances(topic_word_distributions)

# Calcolo della media per ciascuna riga
row_means = np.mean(distance_matrix, axis=1)

# Visualizzazione della matrice delle distanze (calcolo della media per riga in cui per ciascuna riga abbiamo le distanze del topic di riferimento)
print("Matrice delle distanze coseno tra i topic:")
row_means

Matrice delle distanze coseno tra i topic:


array([0.599455  , 0.5262836 , 0.6733517 , 0.72038656, 0.6281475 ,
       0.8003634 , 0.64342   , 0.7863159 , 0.6528545 , 0.58879995],
      dtype=float32)

Come possiamo vedere mediamente i 10 Topic individuati rilevano una distanza in media tra loro prossima che oscilla tra 0.5 e 0.8.

# Effettuare il Named Entity Recognition sul nome delle Organizzazioni

Tra le mail non Spam del dataset andiamo a individuare tutti i nomi delle Organizzazioni in esso contenute.

Prima di tutto vado a caricarmi il modello

In [61]:
import spacy

nlp = spacy.load('en_core_web_sm') #Mi import spacy e mi carico il modello

Andiamo ora a filtrarci i testi NON SPAM:

In [64]:
df_no_spam = df[df['label_num']==0]

In [70]:
sentences_no_spam = df['text']

Partendo da esse applicheremo il Named Entity Recognition (NER) sulle organizzazioni (ORG):

In [74]:
# Lista per memorizzare le organizzazioni trovate
list_org = []

# Iterazione attraverso le frasi
for elem in sentences_no_spam:
    doc = nlp(elem)
    for token in doc.ents:
        if token.label_ == 'ORG':
            #print(f"{token.text} - {token.label_}")
            list_org.append(token.text)

# Visualizzazione delle organizzazioni trovate
print("\nOrganizzazioni trovate:")
print(list_org)

, inc . - ORG
the
securities exchange act - ORG
odin - ORG
transactions inc . - ORG
the
house - ORG
transactions inc . - ORG
transactions inc . - ORG
transactions inc . - ORG
the securities exchange act - ORG
sec - ORG
gas , inc . - ORG
alberta
canada - ORG
gas inc . - ORG
mogi - news - ORG
the
mississippian lower shunda - ORG
ps / 2 - ORG
usb - ORG
modem
$ - ORG
intel - ORG
kb - ORG
samsung - ORG
toshiba - ORG
gsm - ORG
modem
notebookplus - ORG
intel - ORG
toshiba - ORG
microsoft - ORG
ibm - ORG
sony - ORG
jebel ali duty free - ORG
congress - ORG
microsoft - ORG
microsoft - ORG
zwftnqlp oaohkv icckmv rmcetdf - ORG
ozaqgcf czjj - ORG
pajrtqy - ORG
microsoft - ORG
microsoft - ORG
pioneer - ORG
| inc . - ORG
ypil join - ORG
dia - ORG
cabie - ORG
ura | state university - ORG
teiephone inc . - ORG
the securities exchange act - ORG
sec - ORG
red am
school - ORG
byte - ORG
intel - ORG
toshiba - ORG
microsoft - ORG
ibm - ORG
sony - ORG
jebel ali duty free - ORG
congress - ORG
techlite , inc .

Exception ignored in: <function _xla_gc_callback at 0x7d5f8ea5e170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


national events , inc . - symbo - ORG
american express - ORG
budweiser - ORG
nbc - ORG
nationa - ORG
events , inc . - ORG
nationa - ORG
match point , inc . - ORG
the internationa | television - ORG
the securities exchange act - ORG
info / 92 - ORG
info / 92 - ORG
lloyd julia - ORG
china world trade corp . - ORG
china world trade corp . - ORG
world trade corp - ORG
the china world trade corporation ' - ORG
the
civil aviation administration of china - ORG
the
administrative bureau of tourism of china - ORG
tian di ' ' - ORG
china world trade corp : - ORG
china world trade corp . - ORG
world
trade corp . - ORG
the world
trade center association - ORG
the guangzhou world trade center club - ORG
the beijing world trade center club - ORG
the china world trade corp executive - ORG
china world trade
corp : - ORG
china world trade corp - ORG
the securities exchange act - ORG
htm computer technologies - ORG
microsoft - ORG
ciallis softabs - ORG
intel - ORG
toshiba - ORG
microsoft - ORG
ibm - ORG

KeyboardInterrupt: 